In [1]:
import PIL
from PIL import Image
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Input,LSTM, Dense, Dropout, Conv2D, MaxPool2D, Flatten, Activation,  MaxPooling2D, BatchNormalization
from keras import regularizers, metrics

In [2]:
def img_t_train_data(img_name, droprate = 0.9, truncate=True):
    image = Image.open(img_name)
    image = image.convert('RGB')
    print(image.format)
    print(image.size)
    print(image.mode)
    data = np.asarray(image)
    img_width = data.shape[0]
    img_height = data.shape[1]
    data = data.astype("float64")/255
    input1 = np.zeros(((img_height-3)*(img_width-3),15,3))
    input2 = np.zeros(((img_height-3)*(img_width-3),2))
    target = np.zeros(((img_height-3)*(img_width-3),3))
    input_i = 0;
    print(data.shape, input1.shape)
    for i in range(3,img_width):
        for j in range(3,img_height):
            input2[input_i,0] = i/img_width
            input2[input_i,1] = j/img_height
            input1[input_i,0]=data[i-3][j-3]
            input1[input_i,1]=data[i-2][j-3]
            input1[input_i,2]=data[i-1][j-3]
            input1[input_i,3]=data[i][j-3]
            input1[input_i,4]=data[i-3][j-2]
            input1[input_i,5]=data[i-2][j-2]
            input1[input_i,6]=data[i-1][j-2]
            input1[input_i,7]=data[i][j-2]
            input1[input_i,8]=data[i-3][j-1]
            input1[input_i,9]=data[i-2][j-1]
            input1[input_i,10]=data[i-1][j-1]
            input1[input_i,11]=data[i][j-1]
            input1[input_i,12]=data[i-3][j]
            input1[input_i,13]=data[i-2][j]
            input1[input_i,14]=data[i-1][j]
            target[input_i] = data[i][j]
            input_i +=1
    # manually reduce the sample size of exactly same pixels as preceding only keeps 10% of uniform case
    delete_list = []
    r = np.random.rand(input1.shape[0])
    for i in range(input1.shape[0]):
        if (input1[i] == input1[i][0]).all() and r[i] < droprate:
            delete_list.append(i)

    input1 = np.delete(input1,delete_list,0)
    input1_flatten = input1.reshape((input1.shape[0],15*3))
    input2 = np.delete(input2,delete_list,0)
    input = np.concatenate((input1_flatten,input2),axis=1)
    target = np.delete(target,delete_list,0)
    input = input.reshape((input.shape[0],47,1))
    if truncate:
        incompletebatch = input.shape[0] % 100
        delete_list = range(input.shape[0]-incompletebatch, input.shape[0])
        input = np.delete(input,delete_list,0)
        target = np.delete(target,delete_list,0)
    print(input.shape, target.shape)
    return (input, target)

In [3]:
inputs = []
targets = []

for i in range(1,11):
    input,target = img_t_train_data("train"+str(i)+".png")
    inputs.append(input)
    targets.append(target)


None
(600, 600)
RGB
(600, 600, 3) (356409, 15, 3)
(112800, 47, 1) (112800, 3)
None
(600, 600)
RGB
(600, 600, 3) (356409, 15, 3)
(110300, 47, 1) (110300, 3)
None
(600, 600)
RGB
(600, 600, 3) (356409, 15, 3)
(98300, 47, 1) (98300, 3)
None
(600, 600)
RGB
(600, 600, 3) (356409, 15, 3)
(93800, 47, 1) (93800, 3)
None
(600, 600)
RGB
(600, 600, 3) (356409, 15, 3)
(86900, 47, 1) (86900, 3)
None
(600, 600)
RGB
(600, 600, 3) (356409, 15, 3)
(81500, 47, 1) (81500, 3)
None
(600, 600)
RGB
(600, 600, 3) (356409, 15, 3)
(112700, 47, 1) (112700, 3)
None
(329, 329)
RGB
(329, 329, 3) (106276, 15, 3)
(39300, 47, 1) (39300, 3)
None
(329, 329)
RGB
(329, 329, 3) (106276, 15, 3)
(44800, 47, 1) (44800, 3)
None
(600, 600)
RGB
(600, 600, 3) (356409, 15, 3)
(136100, 47, 1) (136100, 3)


In [4]:
model = Sequential()
model.add(keras.layers.InputLayer(batch_input_shape=(100,47,1)))
model.add(LSTM(256,stateful=True))
model.add(Dense(15*16*3, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(15*16*3, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(5*4*3, activation="relu"))
model.add(Dense(3, activation="relu"))
model.compile(loss=tf.keras.losses.MeanAbsoluteError(), metrics=[metrics.MeanAbsoluteError()], optimizer='adam')

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (100, 256)                264192    
_________________________________________________________________
dense (Dense)                (100, 720)                185040    
_________________________________________________________________
dropout (Dropout)            (100, 720)                0         
_________________________________________________________________
dense_1 (Dense)              (100, 720)                519120    
_________________________________________________________________
dropout_1 (Dropout)          (100, 720)                0         
_________________________________________________________________
dense_2 (Dense)              (100, 60)                 43260     
_________________________________________________________________
dense_3 (Dense)              (100, 3)                  1

In [6]:
valid_in, valid_target = img_t_train_data("test3.png",droprate=0)
for j in range(10):
    for i in range(0, len(inputs)):
        model.fit(inputs[i], targets[i], batch_size=100,epochs=1, validation_data=(valid_in, valid_target))
        model.reset_states()

None
(600, 600)
RGB
(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
1361/1361 [==============================] - 15s 11ms/step - loss: 0.0208 - mean_absolute_error: 0.0208 - val_loss: 0.0923 - val_mean_absolute_error: 0.0923


In [7]:
predictmodel = Sequential()
predictmodel.add(keras.layers.InputLayer(batch_input_shape=(1,47,1)))
predictmodel.add(LSTM(256,stateful=True))
predictmodel.add(Dense(15*16*3, activation="relu"))
predictmodel.add(Dropout(0.1))
predictmodel.add(Dense(15*16*3, activation="relu"))
predictmodel.add(Dropout(0.1))
predictmodel.add(Dense(5*4*3, activation="relu"))
predictmodel.add(Dense(3, activation="relu"))
predictmodel.compile(loss=tf.keras.losses.MeanAbsoluteError(), metrics=[metrics.MeanAbsoluteError()], optimizer='adam')
predictmodel.set_weights(model.get_weights())

In [8]:
#  return the residual image
# def predict_img(img_name, model):
img_name = "test2.png"
image = Image.open(img_name)
image = image.convert('RGB')
data = np.asarray(image)
img_width = data.shape[0]
img_height = data.shape[1]
data = data.astype("float64")/255
img_input,target = img_t_train_data(img_name,droprate=0,truncate=False)
model.reset_states()
result = predictmodel.predict(img_input, batch_size=1)
print (result.shape)

# return residual

None
(600, 600)
RGB
(600, 600, 3) (356409, 15, 3)
(356409, 47, 1) (356409, 3)
(356409, 3)


In [9]:
result = result.reshape((img_width-3,img_height-3,3))
resultint = result*255
resultint = np.rint(resultint)
resultint.clip(0,255)
data = np.asarray(image).astype("float64")
residual = data[3:,3:] - resultint
print(np.average(np.absolute(residual)))

18.831183649496317


In [ ]:
data[3:,3:] = residual
datachunk = np.rint(data).astype("int")


In [12]:
with open("datachunk.npy", 'wb') as f:
    np.save(f,datachunk)


In [13]:
with open("datachunk.npy", 'rb') as f:
    a = np.load(f)

In [18]:
from huffman import huffman 
# total_bits = int(huffman(np.asarray(image).astype("float64").flatten()))
total_bits = int(huffman(datachunk.flatten()))
print('total: ', total_bits)
result2 = result

 Char | Huffman code 
----------------------
 0    |           1
 30   |        1010
freq:  (30, 61071)
 33   |        1011
freq:  (33, 55797)
 35   |        1000
freq:  (35, 54534)
 13   |       11010
freq:  (13, 44795)
 21   |       11011
freq:  (21, 43770)
 17   |       11001
freq:  (17, 35828)
 -13  |       10010
freq:  (-13, 34136)
 6    |       10011
freq:  (6, 29922)
 15   |     1111110
freq:  (15, 25483)
 14   |     1111111
freq:  (14, 25330)
 12   |     1111101
freq:  (12, 25008)
 16   |    11111000
freq:  (16, 23243)
 20   |    11111001
freq:  (20, 22018)
 8    |    11110110
freq:  (8, 15987)
 11   |    11110111
freq:  (11, 15588)
 4    |    11110100
freq:  (4, 15423)
 9    |    11110101
freq:  (9, 13978)
 22   |    11110010
freq:  (22, 13761)
 29   |    11110011
freq:  (29, 13526)
 18   |    11110000
freq:  (18, 13369)
 19   |    11110001
freq:  (19, 13266)
 23   |    11101110
freq:  (23, 12947)
 24   |    11101111
freq:  (24, 11460)
 5    |    11101100
freq:  (5, 11262)
 -2

In [19]:
#  return the residual image
# def predict_img(img_name, model):
img_name = "test1.png"
image = Image.open(img_name)
image = image.convert('RGB')
data = np.asarray(image)
img_width = data.shape[0]
img_height = data.shape[1]
data = data.astype("float64")/255
img_input,target = img_t_train_data(img_name,droprate=0,truncate=False)
model.reset_states()
result = predictmodel.predict(img_input, batch_size=1)
print (result.shape)
result = result.reshape((img_width-3,img_height-3,3))
resultint = result*255
resultint = np.rint(resultint)
resultint.clip(0,255)
data = np.asarray(image).astype("float64")
residual = data[3:,3:] - resultint
print(np.average(np.absolute(residual)))
data[3:,3:] = residual
datachunk = np.rint(data).astype("int")
total_bits = int(huffman(datachunk.flatten()))
print('total: ', total_bits)
result1 = result

None
(600, 600)
RGB
(600, 600, 3) (356409, 15, 3)
(356409, 47, 1) (356409, 3)
(356409, 3)
19.292270023110152
 Char | Huffman code 
----------------------
 24   |          00
freq:  (24, 157935)
 25   |          01
freq:  (25, 118153)
 23   |          11
freq:  (23, 106863)
 26   |        1010
freq:  (26, 66847)
 21   |       10111
freq:  (21, 48179)
 0    |      100100
freq:  (0, 35138)
 20   |      100010
freq:  (20, 34780)
 22   |      100011
freq:  (22, 33697)
 27   |      100001
freq:  (27, 30961)
 19   |    10110110
freq:  (19, 24334)
 18   |    10110111
freq:  (18, 22867)
 6    |    10110100
freq:  (6, 22328)
 28   |    10110101
freq:  (28, 18974)
 15   |    10110010
freq:  (15, 18135)
 17   |    10110011
freq:  (17, 16671)
 4    |    10110000
freq:  (4, 15688)
 16   |    10110001
freq:  (16, 14737)
 14   |    10011110
freq:  (14, 14339)
 10   |    10011111
freq:  (10, 13309)
 13   |    10011100
freq:  (13, 13145)
 3    |    10011101
freq:  (3, 12615)
 12   |    10011011
freq:  (

In [20]:
#  return the residual image
# def predict_img(img_name, model):
img_name = "test3.png"
image = Image.open(img_name)
image = image.convert('RGB')
data = np.asarray(image)
img_width = data.shape[0]
img_height = data.shape[1]
data = data.astype("float64")/255
img_input,target = img_t_train_data(img_name,droprate=0,truncate=False)
model.reset_states()
result = predictmodel.predict(img_input, batch_size=1)
print (result.shape)
result = result.reshape((img_width-3,img_height-3,3))
resultint = result*255
resultint = np.rint(resultint)
resultint.clip(0,255)
data = np.asarray(image).astype("float64")
residual = data[3:,3:] - resultint
print(np.average(np.absolute(residual)))
data[3:,3:] = residual
datachunk = np.rint(data).astype("int")
total_bits = int(huffman(datachunk.flatten()))
print('total: ', total_bits)
result3 = result

None
(600, 600)
RGB
(600, 600, 3) (356409, 15, 3)
(356409, 47, 1) (356409, 3)
(356409, 3)
23.567017106750953
 Char | Huffman code 
----------------------
 0    |           1
 19   |        1010
freq:  (19, 49233)
 18   |        1000
freq:  (18, 49172)
 17   |        1001
freq:  (17, 48673)
 16   |       11110
freq:  (16, 31788)
 1    |       11100
freq:  (1, 30434)
 -1   |       11101
freq:  (-1, 28437)
 15   |       11011
freq:  (15, 27313)
 -2   |     1111110
freq:  (-2, 24499)
 5    |     1111111
freq:  (5, 23729)
 -3   |     1111100
freq:  (-3, 21407)
 2    |     1111101
freq:  (2, 21167)
 20   |     1101010
freq:  (20, 20719)
 -85  |     1101000
freq:  (-85, 20495)
 -84  |     1101001
freq:  (-84, 20365)
 6    |     1100110
freq:  (6, 19742)
 4    |     1100111
freq:  (4, 18801)
 8    |     1100100
freq:  (8, 16513)
 -4   |     1100101
freq:  (-4, 16086)
 3    |     1100010
freq:  (3, 16072)
 -83  |     1100011
freq:  (-83, 14947)
 7    |     1100001
freq:  (7, 13497)
 -5   |    1